# Spam Email Detection Using Deep Learning

### Load Libraries Required

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import re
import os
from sklearn.preprocessing import LabelEncoder
import tensorflow_hub as hub
import tensorflow_text as text

### Load Dataset

In [2]:
np.set_printoptions(precision=3,suppress=True)

# Make your numpy data readable

In [3]:
# Load data using pd

data = pd.read_csv("Dataset.csv")
data.head()


v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8  \
0        NaN        NaN        NaN        NaN        NaN        NaN   
1        NaN        NaN        NaN        NaN        NaN        NaN   
2        NaN        NaN        NaN        NaN        NaN        NaN   
3        NaN        NaN        NaN        NaN        NaN        NaN   
4        NaN        NaN        NaN        NaN        NaN        NaN   

  Unnamed: 9  ... Unnamed: 28 Unnamed: 29 Unnamed: 30 Unnamed: 31 Unnamed: 32  \
0        NaN  ...         NaN         NaN         NaN         NaN         NaN   
1        NaN  ...         NaN         NaN         NaN         NaN         NaN   
2        NaN  ...         NaN         NaN         NaN         NaN         NaN   
3        NaN  ...         NaN         NaN         NaN         NaN         NaN   
4        NaN  ...         NaN         NaN         NaN         NaN         NaN   

  Unnamed: 33 Unnamed: 34 Unnamed: 35 Unnamed: 36 Unnamed: 37  
0         NaN         NaN         NaN         NaN         NaN  
1         NaN         NaN         NaN         NaN         NaN  
2         NaN         NaN         NaN         NaN         NaN  
3         NaN         NaN         NaN         NaN         NaN  
4         NaN         NaN         NaN         NaN         NaN  

[5 rows x 38 columns]

### Data Preprocessing

In [4]:
# Few unwanted columns got added to dataset, so just extract 
# the required columns from it.
df =  pd.DataFrame()
df[['label','text']] = data[['v1','v2']]



In [5]:
df.head()

label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

In [6]:
df.describe()
# Total record in are 5567 as we are going to drop one null record 

label                    text
count   5568                    5567
unique     2                    5161
top      ham  Sorry, I'll call later
freq    4821                      30

In [7]:


df.groupby('label').describe()



text                                                               
      count unique                                                top freq
label                                                                     
ham    4820   4506                             Sorry, I'll call later   30
spam    747    655  Please call our customer service representativ...    4

In [8]:
df.isna().sum()

label    0
text     1
dtype: int64

In [9]:
# drop null values

df.dropna(inplace= True)
df.isna().sum()

label    0
text     0
dtype: int64

In [10]:
df.head()

label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

In [11]:
min_len = df['text'].min()
print(len(min_len))\

max_len = df['text'].max()
print(len(max_len))

21
81


In [12]:
df['text']

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
5       FreeMsg Hey there darling it's been 3 week's n...
6       Even my brother is not like to speak with me. ...
7       As per your request 'Melle Melle (Oru Minnamin...
8       WINNER!! As a valued network customer you have...
9       Had your mobile 11 months or more? U R entitle...
10      I'm gonna be home soon and i don't want to tal...
11      SIX chances to win CASH! From 100 to 20,000 po...
12      URGENT! You have won a 1 week FREE membership ...
13      I've been searching for the right words to tha...
14                    I HAVE A DATE ON SUNDAY WITH WILL!!
15      XXXMobileMovieClub: To use your credit, click ...
16                             Oh k...i'm watching here:)
17      Eh u r

In [13]:
# Spit up x and y
x = df['text'].copy()
y = df['label'].copy()

print(x.head())
print(y.head())

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: text, dtype: object
0     ham
1     ham
2    spam
3     ham
4     ham
Name: label, dtype: object


In [14]:
# Lets do encoding on target dataset i.e. y

In [15]:
y.head()

0     ham
1     ham
2    spam
3     ham
4     ham
Name: label, dtype: object

In [16]:
le = LabelEncoder()
y_enc = le.fit_transform(y)

In [17]:
y_encoded = pd.DataFrame(y_enc)
# ham - 0 and spam-1

In [18]:
y_encoded.head()

0
0  0
1  0
2  1
3  0
4  0

In [19]:
# Now we have y_encoded and x as our input for further process

In [20]:
x.shape

(5567,)

In [21]:
y_encoded.shape

y_encoded.columns = ['label']

In [22]:
y_encoded.head()

label
0      0
1      0
2      1
3      0
4      0

In [23]:
x_df = pd.DataFrame(x)

In [24]:
x_df.head()

text
0  Go until jurong point, crazy.. Available only ...
1                      Ok lar... Joking wif u oni...
2  Free entry in 2 a wkly comp to win FA Cup fina...
3  U dun say so early hor... U c already then say...
4  Nah I don't think he goes to usf, he lives aro...

In [25]:
X = x_df.copy()
Y = y_encoded.copy()

In [26]:
X.head()

text
0  Go until jurong point, crazy.. Available only ...
1                      Ok lar... Joking wif u oni...
2  Free entry in 2 a wkly comp to win FA Cup fina...
3  U dun say so early hor... U c already then say...
4  Nah I don't think he goes to usf, he lives aro...

In [27]:
Y.head()

label
0      0
1      0
2      1
3      0
4      0

In [28]:
print(X.shape)
print(Y.shape)

#  size of X and Y is same

(5567, 1)
(5567, 1)


In [29]:
# Lets do the spitting of data for train and test

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain,ytest = train_test_split(X,Y,test_size=0.2)


In [30]:
print(xtrain.shape)
print(xtest.shape)

print(ytrain.shape)
print(ytest.shape)

(4453, 1)
(1114, 1)
(4453, 1)
(1114, 1)


In [31]:
# BERT model 1. preprocessor and encoder are trained here
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3",
    trainable=True)

In [32]:
# Lets create BERT model for text embedding
# Lets start using keras for text preprocessing and vectorization

def sentence_embedding(sentence):
    preprocessed_input = preprocessor(sentence)
    encoded_input = encoder(preprocessed_input)
    return encoded_input['pooled_output']

In [33]:
# Test out function for correct output
sentence_embedding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.844, -0.513, -0.888, ..., -0.747, -0.753,  0.92 ],
       [-0.872, -0.505, -0.944, ..., -0.858, -0.717,  0.881]],
      dtype=float32)>

In [34]:
# Bert Layer

text_input = tf.keras.layers.Input(shape=(),dtype=tf.string,name='text')
preprocessed_text = preprocessor(text_input)
output = encoder(preprocessed_text)

In [35]:
# Neural Network Layer

l = tf.keras.layers.Dropout(0.1, name="dropout") (output['pooled_output'])

l = tf.keras.layers.Dense(1, activation='sigmoid',name = 'output')(l)

In [36]:
# use i/p and op for model

model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [37]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_word_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'encoder_outputs':  109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
_______________________________________________________________________________________

In [38]:
METRICS = [
    tf.keras.metrics.Accuracy(name="accuracy"),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall")
]

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics= METRICS
    )

In [1]:
# train the model

model.fit(xtrain, ytrain, epochs=1)

NameError: name 'model' is not defined